## Asset replication

Asset replication can be used to hedging or mean-reversion trading purposes.

In [ ]:
import pod
import pandas
import zipfile
from io import StringIO
from typing import List
from datetime import date

path_data = 'data'
daily_data = []
for year in range(2018, 2023):
    with zipfile.ZipFile(f"{path_data}/NYSE_{year}.zip", "r") as zip_file:
        for file in zip_file.namelist():
            with zip_file.open(file, "r") as zip_entry:
                current_data = zip_entry.read().decode('utf-8')
                current_day = pandas.read_csv(StringIO(current_data), dtype={
                    'Symbol': 'string',
                    'Date': 'string',
                    'Open': 'float64',
                    'High': 'float64',
                    'Close': 'float64',
                    'Volume': 'int64'
                }, parse_dates=['Date'], date_format="%d-%b-%Y")
                daily_data.append(current_day)

market_data = pandas.concat(daily_data)
prices = market_data[['Date', 'Symbol', 'Close']].pivot_table(index='Date', columns='Symbol', values='Close').ffill()
from datetime import date
train_prices = prices.loc[prices.index < pandas.to_datetime(date(2022, 1, 1))]
test_prices = prices.loc[prices.index >= pandas.to_datetime(date(2022, 1, 1))]

performances = train_prices.pct_change().dropna(how='all', axis=0).dropna(how='any', axis=1)
